In [57]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diefesson\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [58]:
INPUT_PATH = "datasets/ruddit_with_text.csv"
OUTPUT_PATH = "datasets/cleaned_dataset.csv"


In [59]:
dataset = pd.read_csv(INPUT_PATH)
dataset.head()

,post_id,comment_id,txt,url,offensiveness_score
0,42g75o,cza1q49,> The difference in average earnings between m...,https://www.reddit.com/r/changemyview/comments...,-0.083
1,42g75o,cza1wdh,"The myth is that the ""gap"" is entirely based o...",https://www.reddit.com/r/changemyview/comments...,-0.022
2,42g75o,cza23qx,[deleted],https://www.reddit.com/r/changemyview/comments...,0.167
3,42g75o,cza2bw8,The assertion is that women get paid less for ...,https://www.reddit.com/r/changemyview/comments...,-0.146
4,42g75o,cza2iji,You said in the OP that's not what they're mea...,https://www.reddit.com/r/changemyview/comments...,-0.083


In [60]:
english_stopwords = stopwords.words("english")
clean_dataset = pd.DataFrame()
clean_dataset["offensiveness_score"] = dataset["offensiveness_score"]
clean_dataset["keywords"] = dataset["txt"] \
    .map(lambda comment: [word.lower() for word in comment.split(" ") if word.isalpha()]) \
    .map(lambda comment: [word for word in comment if word not in english_stopwords])
clean_dataset.head()

,offensiveness_score,keywords
0,-0.083,"[difference, average, earnings, men, women, ex..."
1,-0.022,"[myth, entirely, based, sex]"
2,0.167,[]
3,-0.146,"[assertion, women, get, paid, less, get, paid,..."
4,-0.083,"[said, op, measuring, women, make, jlaw, compl..."


In [61]:
clean_dataset.to_csv(OUTPUT_PATH)